In [2]:
import pandas as pd

In [8]:
# 주요 파일 경로
move_path = "../data/VL_csv/tn_move_his_이동내역_Cleaned_E.csv"
visit_area_path = "../data/VL_csv/tn_visit_area_info_방문지정보_Cleaned_E_mapped.csv"
photo_path = "../data/VL_csv/tn_tour_photo_관광사진_E.csv"

# 데이터 로딩
move_df = pd.read_csv(move_path)
photo_df = pd.read_csv(photo_path)
visit_area_df = pd.read_csv(visit_area_path)

In [9]:
# visit_area_df의 visit_area_nm 기반으로 새로운 고유 ID 부여
unique_visit_area_nms = visit_area_df["VISIT_AREA_NM"].unique()

In [10]:
visit_area_df.shape

(15941, 23)

In [11]:
visit_area_nm_to_new_id = {nm: idx for idx, nm in enumerate(unique_visit_area_nms)}

In [12]:
visit_area_nm_to_new_id

{'화성 관광열차 안내소 연무대 매표소': 0,
 '창룡문': 1,
 '수원 화성 화홍문': 2,
 '청량리역 경춘선': 3,
 '청평역 경춘선': 4,
 '경춘선 자전거길': 5,
 '베이커리 시어터': 6,
 '백암천': 7,
 '자시오 잣 주꾸미': 8,
 '신창현 풍림아이원 1차 아파트': 9,
 '마석역 경춘선': 10,
 '청량리역': 11,
 '농협안성팜랜드': 12,
 '스타필드 안성': 13,
 '대동국수 평택역점': 14,
 '편지': 15,
 '호텔 프라하': 16,
 '평택 초등학교': 17,
 '샤오바오 고덕점': 18,
 '평택시 농업 생태원': 19,
 '평택시 로컬푸드 재단': 20,
 '시흥 하늘휴게소 양방향': 21,
 '영남아파트': 22,
 '충무아트센터 대극장': 23,
 '스탠더드 커피': 24,
 '연신내 칼국수': 25,
 '독립문': 26,
 '롯데월드 샤롯데씨어터': 27,
 '미사리밀빛초계국수 롯데월드점': 28,
 '사무실': 29,
 '서울역 1호선': 30,
 '삼환아파트': 31,
 '인사동 문화의 거리': 32,
 '꽃밥에 피다': 33,
 '후후 라운지': 34,
 'IFC 몰': 35,
 '서울랜드, 국립현대미술관': 36,
 '동원 홈 푸드라운지 국립 현대미술관점': 37,
 '광명역': 38,
 '삼진 어묵 부산역 광장점': 39,
 '김해 율하시티프라디움아파트': 40,
 '정안 알밤 휴게소 천안 방향': 41,
 '파주 프리미엄 아울렛': 42,
 '이마트 파주점': 43,
 '해브펀 캠핑 어라운드': 44,
 '이케아 광명점': 45,
 '지중해마을': 46,
 '에버랜드': 47,
 '스타필드 코엑스몰': 48,
 '본가': 49,
 '서울어린이대공원': 50,
 '스타벅스 아차산역': 51,
 '명륜 진사갈비 서울 중곡점': 52,
 '에인절 코인 노래연습장 건대점': 53,
 '코지에 FunCity': 54,
 '엑스 클라임 건대점': 55,
 '아웃 닭 건대점': 56,
 '서피 동파

In [ ]:
visit_area_df["NEW_VISIT_AREA_ID"] = visit_area_df["VISIT_AREA_NM"].map(visit_area_nm_to_new_id)

# photo_df의 VISIT_AREA_NM 결측값 채우기
photo_df = photo_df.merge(
    visit_area_df[["VISIT_AREA_ID", "TRAVEL_ID", "VISIT_AREA_NM"]],
    how="left",
    on=["VISIT_AREA_ID", "TRAVEL_ID"],
    suffixes=("", "_filled")
)
photo_df["VISIT_AREA_NM"] = photo_df["VISIT_AREA_NM"].fillna(photo_df["VISIT_AREA_NM_filled"])
photo_df.drop(columns=["VISIT_AREA_NM_filled"], inplace=True)

# visit_area_id -> visit_area_nm 매핑 생성
visit_area_id_to_nm = visit_area_df.set_index("VISIT_AREA_ID")["VISIT_AREA_NM"].to_dict()

# move_df의 start/end_visit_area_id를 새로운 id로 변환
move_df["START_VISIT_AREA_NM"] = move_df["START_VISIT_AREA_ID"].map(visit_area_id_to_nm)
move_df["END_VISIT_AREA_NM"] = move_df["END_VISIT_AREA_ID"].map(visit_area_id_to_nm)
move_df["START_NEW_ID"] = move_df["START_VISIT_AREA_NM"].map(visit_area_nm_to_new_id)
move_df["END_NEW_ID"] = move_df["END_VISIT_AREA_NM"].map(visit_area_nm_to_new_id)

# photo_df에도 새로운 id 컬럼 생성
photo_df["NEW_VISIT_AREA_ID"] = photo_df["VISIT_AREA_NM"].map(visit_area_nm_to_new_id)

# 결과 저장
visit_area_df.to_csv("../data/VL_csv/visit_area_with_new_id_final.csv", index=False)
move_df.to_csv("../data/VL_csv/move_with_new_id_final.csv", index=False)
photo_df.to_csv("../data/VL_csv/photo_with_new_id_final.csv", index=False)

# 결과 요약
{
    "visit_area_df_shape": visit_area_df.shape,
    "move_df_shape": move_df.shape,
    "photo_df_shape": photo_df.shape,
    "sample_visit_area_df": visit_area_df.head(2).to_dict(),
    "sample_move_df": move_df.head(2).to_dict(),
    "sample_photo_df": photo_df.head(2).to_dict()
}